## Importar librerías

In [3]:
import numpy as np

## Definir clase para algoritmo SVM

### Constructor

Se definen las variables relevantes para el algoritmo:
- *learning rate*: Parámetro que define el tamaño de las correcciones de los pesos en entrenamiento.
- *lambda param*: Parámetro que define la importancia relativa entre minimizar los pesos y reducir el número de instancias mal clasificadas y penalizadas.
- *n_iters*: Número de iteraciones para el entrenamiento.
- *w*: Pesos del modelo, que definen el hiperplano para realizar la clasificación.
- *b*: Constante del modelo que define donde el hiperplano cruza el origen.

### Fit

Función para ajustar los pesos del modelo, que definen el hiperplano para separar los datos y realizar la clasificación.
Se usa el método del gradiente descendiente para el ajuste de pesos.

- Se buscan los pesos *w* tal que se cumpla:

$$f(x_i) =
    \begin{cases}
    w\cdot x_i + b \ge 0 & si \space y_i = +1 & \\
    w\cdot x_i + b < 0 & si \space y_i = -1
    \end{cases}$$
    

- Lo que se puede resumir en la ecuación:

$$  y_i \cdot h(x_i) = y_i(w\cdot x_i+b) \ge 1$$

- Donde

$$\space h(x_i) = w\cdot x_i + b$$

- Se quiere minimizar los pesos para reducir la distancia entre los *vectores de soporte*, lo que se describe como:

$$ l(w)= \lambda\|w\|^2 $$


- A la vez, se debe penalizar a las predicciones equivocadas:

$$l_i =
    \begin{cases}
    0 & si \space y_i \cdot h(x_i) \ge 1 & \\
    1 - y_i \cdot f(x_i) & si \, no
    \end{cases}$$

- Juntando ambas condiciones, se busca minimizar la función de costo:

$$J_i =
    \begin{cases}
    \lambda \|w\|^2 & si \space y_i \cdot h(x_i) \ge 1 & \\
    \lambda \|w\|^2 + 1-y_i(w\cdot x_i - b) & si \, no
    \end{cases}$$
    
- Calculando la derivada parcial con respecto a los pesos (para usar el algoritmo de gradiente descendiente) se tiene:

$$\frac{\partial J_i}{\partial w_k} =
    \begin{cases}
    2 \lambda w_k & si \space y_i \cdot h(x_i) \ge 1 & \\
    2 \lambda w_k -y_i\cdot x_{ik} & si \, no
    \end{cases}$$
    

### Predict

Función para predecir clase del dato de entrada.

- Para obtener la predicción del modelo sobre una instancia $x_i$ basta calcular:

$$h(x_i) = w\cdot x_i + b$$

- Y la predicción del modelo es:

$$y_i =
    \begin{cases}
    1 & si \space h(x_i) \ge 0 & \\
    0 & si \space h(x_i) < 0
    \end{cases}$$


In [1]:
class SVM:

    def __init__(self, learning_rate=0.001, lambda_param=0.001, n_iters=1000):
        self.lr = learning_rate
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Pasar la etiquetas de 0 y 1 a las etiquetas -1 y 1
        y_ = np.where(y <= 0, -1, 1)

        # Inicializar pesos
        self.w = np.random.random((n_features))
        self.b = 0

        # Por cada iteración de entrenamiento
        for _ in range(self.n_iters):
            # Por cada datapoint de entrenamiento
            for idx, x_i in enumerate(X):
                # Verificar la condición 𝑦 ⋅ ℎ(𝑥) ≥ 1
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # Usar el gradiente descendiente 2𝜆𝑤
                    self.w -= self.lr * (2 * self.lambda_param * self.w)
                else:
                    # Usar el gradiente descendiente 2𝜆𝑤 − 𝑦 ⋅ 𝑥
                    self.w -= self.lr * (2 * self.lambda_param * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]


    def predict(self, X):
        # Calcular ℎ(𝑥) = 𝑤 ⋅ 𝑥 - 𝑏, 
        approx = np.dot(X, self.w) - self.b
        # Devolver 1 si ℎ(𝑥) > 0 y devolver 0 si ℎ(𝑥) < 0
        return (approx >= 0).astype(int)

## Análisis de resultados

### Cálculo de *accuracy*

In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return str(100*accuracy)[0:5] + "%"

### Visualización de resultados

- Se grafica la intersección del hiperplano para la clasificación.
- Se grafican los *vectores de soporte*.
- Se grafican los *datapoints*, coloreandolos según su etiqueta real.

> Notar que se asume que los datos tienen dos dimensiones.

In [ ]:
def visualize_svm(X, y, w, b, title, x_label, y_label, name):
    def get_hyperplane_value(x, w, b, offset):
        return (-w[0] * x + b + offset) / w[1]

    fig = plt.figure()    
    ax = fig.add_subplot(1, 1, 1)
    # Graficar datapoints
    plt.scatter(X[:, 0], X[:, 1], marker="o", c=y, cmap=plt.cm.coolwarm)
    
    # Metadata gráfico
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    x0_1 = np.amin(X[:, 0])
    x0_2 = np.amax(X[:, 0])

    # Intersección del hiperplano
    x1_1 = get_hyperplane_value(x0_1, w, b, 0)
    x1_2 = get_hyperplane_value(x0_2, w, b, 0)

    # Línea donde función del hiperplano es igual a -1 (vector de soporte)
    x1_1_m = get_hyperplane_value(x0_1, w, b, -1)
    x1_2_m = get_hyperplane_value(x0_2, w, b, -1)

    # Línea donde función del hiperplano es igual a +1 (vector de soporte)
    x1_1_p = get_hyperplane_value(x0_1, w, b, 1)
    x1_2_p = get_hyperplane_value(x0_2, w, b, 1)

    #Graficar linea de intersección y vectores de soporte
    ax.plot([x0_1, x0_2], [x1_1, x1_2], "y--")
    ax.plot([x0_1, x0_2], [x1_1_m, x1_2_m], "k")
    ax.plot([x0_1, x0_2], [x1_1_p, x1_2_p], "k")

    x1_min = np.amin(X[:, 1])
    x1_max = np.amax(X[:, 1])
    ax.set_ylim([x1_min - 3, x1_max + 3])
    
    plt.savefig("images/" + name + ".svg")
    
    plt.show()